# 類似文獲得アプローチ検証ノート

文単位で類似文を獲得するアプローチが、果たして考察に役にたつのか？ということを考える。
そのために文単位で類似文を探索する。文書単位ではなく、文単位である。

In [1]:
import jellyfish
import json
import itertools
from typing import List, Dict, Tuple
from joblib import Parallel, delayed

In [2]:
path_resource_json = "../resources/cleaned_text.json"
seq_original_document = json.loads(open(path_resource_json, "r").read())

In [3]:
def split_into_sentence(document_obj: str)->List[Tuple[str, str]]:
    """１作品の文書を句点で区切って、文単位とする。
    作品の出展を残しておくために、タイトル情報も残して返す。
    
    * Return
    - - -
    - list
        - [ {"title": str, "sentence": str} ]
    """
    return [(document_obj["title"], doc) for doc in document_obj["body"].split("。")]


def get_similarity(pair_input_sentence: Tuple[Tuple[str, str], Tuple[str, str]], func_get_similarity)->Tuple[Tuple[str,str],Tuple[str,str],float]:
    """類似度関数をラップするための関数
    
    * Parameters
    - - -
    - func_get_similarity: function
        - function(a, b)を引数にとり、類似度(float)を返す関数
    """
    similarity_score = func_get_similarity(pair_input_sentence[0][1], pair_input_sentence[1][1])
    return pair_input_sentence[0], pair_input_sentence[1], similarity_score


def multi_process(sentence_pair_list: List[Tuple[Tuple[str, str], Tuple[str, str]]], func_get_similarity) -> List[Tuple[Tuple[str,str],Tuple[str,str],float]]:
    """
    文類似度をマルチプロセス(joblib)で計算
    
    * Parameters
    - - -
    - sentence_pair_list: list
        - [ ( (title-a, sentence-a), (title-b, sentence-b) ) ]
    - func_get_similarity: function
        - function(a, b)を引数にとり、類似度(float)を返す関数
    """
    return Parallel(n_jobs=-1)([delayed(get_similarity)(sent_pair) for sent_pair in sentence_pair_list])


In [8]:
seq_sentence_obj = list(itertools.chain.from_iterable([split_into_sentence(original_doc) for original_doc in seq_original_document]))
len(seq_sentence_obj)

10249

In [10]:
# これでペアを作ると恐ろしいことになるので、方策を考える。まずは文長で制御することを考えてみる #
seq_sentence_obj[:50]

[('南島譚 幸福', '昔、此の島に一人の極めて哀れな男がいた'),
 ('南島譚 幸福', '年齢を数えるという不自然な習慣が此の辺には無いので、幾歳ということはハッキリ言えないが、余り若くないことだけは確かであった'),
 ('南島譚 幸福', '髪の毛が余り縮れてもおらず、鼻の頭がすっかり潰れてもおらぬので、此の男の醜貌は衆人の顰笑の的となっていた'),
 ('南島譚 幸福', 'おまけに脣が薄く、顔色にも見事な黒檀の様な艶が無いことは、此の男の醜さを一層甚だしいものにしていた'),
 ('南島譚 幸福', '此の男は、恐らく、島一番の貧乏人であったろう'),
 ('南島譚 幸福', 'ウドウドと称する勾玉の様なものがパラオ地方の貨幣であり、宝であるが、勿論、此の男はウドウドなど一つも持ってはいない'),
 ('南島譚 幸福', 'ウドウドも持っていない位だから、之によって始めて購うことの出来る妻をもてる訳がない'),
 ('南島譚 幸福', 'たった独りで、島の第一長老の家の物置小舎の片隅に住み、最も卑しい召使として仕えている'),
 ('南島譚 幸福', '家中のあらゆる卑しい勤めが、此の男一人の上に負わされる'),
 ('南島譚 幸福', '怠け者の揃った此の島の中で、此の男一人は怠ける暇が無い'),
 ('南島譚 幸福', '朝はマンゴーの繁みに囀る朝鳥よりも早く起きて漁に出掛ける'),
 ('南島譚 幸福', '手槍で大蛸を突き損って胸や腹に吸い付かれ、身体中腫れ上ることもある'),
 ('南島譚 幸福', '巨魚タマカイに追われて生命からがら独木舟に逃げ上ることもある'),
 ('南島譚 幸福', '盥ほどもある車渠貝に足を挟まれ損ったこともある'),
 ('南島譚 幸福',
  '午になり、島中の誰彼が木蔭や家の中の竹床の上でうつらうつら午睡をとる時も、此の男ばかりは、家内の清掃に、小舎の建築に、椰子蜜採りに、椰子縄綯いに、屋根葺きに、家具類の製作に、目が廻る程忙しい'),
 ('南島譚 幸福', '此の男の皮膚はスコールの後の野鼠の様に絶えず汗でびっしょり濡れている'),
 ('南島譚 幸福', '昔から女の仕事と極められている芋田の手入の外は、何から何迄此の男が一人で働く'),
 ('南島譚 幸福',
  '陽が西の海に入って、麺麭の大樹の

In [ ]:
# 2つ組のペアを作成する
#_seq_pair_input_doc = itertools.combinations(seq_sentence_obj, 2)
#  同じ作品のペアは排除する
#seq_pair_input_doc = [t_pair_title_sentence for t_pair_title_sentence in _seq_pair_input_doc if not t_pair_title_sentence[0][0] == t_pair_title_sentence[1][0]]

In [ ]:
#seq_result_similarity_jaro_distance = multi_process(seq_pair_input_doc, jellyfish.jaro_distance)

In [ ]:
#len(seq_pair_input_doc)